In [513]:
### notes

### 2016-0728-R exists in annual report and data nola gov but not brady table: 
### Officer allegedly committed a simple battery on an arrested subject.
### Disposition is dismissed, which presumably means fired. 
### this is an issue because officers can be re-hired after being fired, in which case their misconduct history is non-existant 
### take christopher durning who was fired, and re-hired a year later: https://www.dropbox.com/scl/fi/prrnecu3e0ji1buxno94d/OHR-Christopher-Durning.pdf?rlkey=x0lxyggtfafrhyqnz42o2ided&st=3x2utd03&dl=0
### tie this to wrongful conviction research

In [514]:
### issue with 2016-009-p. Where did this come from?

In [515]:
import pandas as pd

from helper import clean_column_names

In [516]:
### clean coded table ###
def filter_coded_tbl(df):
    df = df[~((df.keep.astype(str) == "0"))]
    return df 

def sanitize_tracking_id(df):
    df.loc[:, "tracking_id"] = (df.tracking_id
                                .str.lower()
                                .str.strip()
                                .str.replace(r"\s+", "", regex=True)
    )
    return df 

### clean brady table ###
def clean_brady_table(df):
    df.columns = [col.lstrip('\'') for col in df.columns]

    df = df.applymap(lambda x: x.lstrip('\'') if isinstance(x, str) else x)
    
    df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)


    df.loc[:, "last_name"] = (df.last_name
                              .str.replace(r"\'RULE(.+)", "", regex=True)
                              .str.replace(r"^\'$", "", regex=True)
    )

    df = df[~((df.last_name.fillna("") == ""))]

    df.loc[:, "tracking_id"] = (df.tracking_id.str
                                .lower()
                                .str.strip()
                                .str.replace(r"\s+", "", regex=True)
                                .str.replace(r"\'", "", regex=True)
    )

    df.loc[:, "allegation"] = (df.allegation_rule
                               .str.cat(df.allegation_paragraph, sep=" ")
    )

    df.loc[:, "allegation"] = (df.allegation
                               .str.lower()
                               .str.strip()
                               .str.replace(r"\'", "", regex=True)
    )

    df = df.drop(columns=["allegation_rule", "allegation_paragraph"])

    df.loc[:, "allegation_desc"] = (df.allegation_desc
                                    .str.replace(r"\'$", "", regex=True)
                                    .str.replace(r"^$", "missing", regex=True)
    )



    return df 

### clean data nola gov tbl ### 
def clean_gov_table(df):
    df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)

    df.loc[:, "tracking_id"] = (df["complaint_tracking_number"].str
                                .lower()
                                .str.strip()
                                .str.replace(r"\s+", "", regex=True)
                                .str.replace(r"\'", "", regex=True)
    )


    return df 


### read tbls ###
def read_coded_tbl():
    df = pd.read_csv("../data/output/coded_data.csv")
    return df 

def read_brady_tbl():
    df = pd.read_csv("../data/input/brady_2016_2023.csv")
    return df 


def read_gov_tbl():
    df = pd.read_csv("../data/input/data_nola_gov.csv")
    return df 


coded_tbl = read_coded_tbl()
coded_tbl = coded_tbl.pipe(sanitize_tracking_id).pipe(filter_coded_tbl)


brady_tbl = read_brady_tbl()
brady_tbl = brady_tbl.pipe(clean_brady_table).pipe(clean_column_names)

gov_tbl = read_gov_tbl()
gov_tbl = gov_tbl.pipe(clean_column_names).pipe(clean_gov_table)

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/2276834004.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lstrip('\'') if isinstance(x, str) else x)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/2276834004.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/2276834004.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if isinstance(x, str) else x)


In [517]:
### find discrepancies in tracking ids (some tracking ids exist in one table but not the others) ###

coded_ids = set(coded_tbl['tracking_id'])
brady_ids = set(brady_tbl['tracking_id'])
gov_ids = set(gov_tbl['tracking_id'])

# Find unique tracking IDs in coded_tbl not in brady_tbl
unique_to_coded_vs_brady = coded_ids.difference(brady_ids)

# Find unique tracking IDs in coded_tbl not in gov_tbl
unique_to_coded_vs_gov = coded_ids.difference(gov_ids)

print(f"Ids in coded table that are not in brady table {unique_to_coded_vs_brady}\nIds in coded table that are not in data nola gov table {unique_to_coded_vs_gov}")

Ids in coded table that are not in brady table {'2016-0481-p', '2016-0752-p', '2017-0720-r', '2019-0322-n', '2021-0464-p', '2020-0177-p', '2019-0423-p', '2017-0115-p', '2020-0232-p', '2016-0157-p', '2017-0215-r', '2016-009-p', '2020-0385-n', '2021-0569-p', '2017-0719-r', '2017-0504-p', '2019-0541-p', '2016-0075-p', '2016-240-p', '2020-0374-p', '2019-0402-p', '2020-0479-p', '2016-0728-r'}
Ids in coded table that are not in data nola gov table {'2017-0504-p', '2020-0529-p', '2021-0499-r', '2016-0157-p', '2017-0215-r', '2016-0752-p', '2019-0050-d', '2016-240-p', '2016-0075-p', '2016-009-p', '2020-0385-n', '2020-0130-n', '2019-0052-d', '2021-0569-p', '2017-0720-r', '2020-0232-p', '2017-0719-r', '2019-0322-n'}


In [518]:
def strip_letter_from_id(df):
    df.loc[:, "tracking_id"] = df.tracking_id.str.replace(r"-(\w{1})$", "", regex=True)
    return df

coded_tbl_clean = coded_tbl.copy()

brady_tbl_clean = brady_tbl.copy()

gov_tbl_clean = gov_tbl.copy()


coded_tbl_clean = coded_tbl_clean.pipe(strip_letter_from_id)
brady_tbl_clean = brady_tbl_clean.pipe(strip_letter_from_id)
gov_tbl_clean = gov_tbl_clean.pipe(strip_letter_from_id)

def find_unique_ids(primary_ids, comparison_ids):
    return primary_ids.difference(comparison_ids)

coded_ids_clean = set(coded_tbl_clean['tracking_id'])
brady_ids_clean = set(brady_tbl_clean['tracking_id'])
gov_ids_clean = set(gov_tbl_clean['tracking_id'])

# Determine relationships between ID sets
coded_not_in_brady = find_unique_ids(coded_ids_clean, brady_ids_clean)
coded_not_in_gov = find_unique_ids(coded_ids_clean, gov_ids_clean)
coded_in_brady_not_in_gov = find_unique_ids(coded_ids_clean.intersection(brady_ids_clean), gov_ids_clean)
coded_in_gov_not_in_brady = find_unique_ids(coded_ids_clean.intersection(gov_ids_clean), brady_ids_clean)
coded_not_in_brady_or_gov = coded_ids_clean.difference(brady_ids_clean.union(gov_ids_clean))

print("Coded IDs not in Brady:", coded_not_in_brady)
print("Coded IDs not in Gov:", coded_not_in_gov)
print("IDs in Coded and Brady but absent from Gov:", coded_in_brady_not_in_gov)
print("IDs in Coded and Gov but absent from Brady:", coded_in_gov_not_in_brady)
print("Coded IDs not in either Brady or Gov:", coded_not_in_brady_or_gov)

# Find common IDs between the sets
coded_and_brady_ids = coded_ids_clean.intersection(brady_ids_clean)
coded_and_gov_ids = coded_ids_clean.intersection(gov_ids_clean)

print("IDs in both Coded and Brady:", coded_and_brady_ids)
print("IDs in both Coded and Gov:", coded_and_gov_ids)

Coded IDs not in Brady: {'2016-009', '2019-0322', '2019-0402', '2021-0569', '2017-0115', '2016-0728', '2016-0481', '2021-0464', '2020-0479', '2019-0423', '2020-0177', '2020-0374', '2019-0541', '2020-0232', '2020-0385', '2016-0752', '2016-240'}
Coded IDs not in Gov: {'2016-009', '2019-0322', '2020-0529', '2021-0569', '2016-240', '2017-0215', '2020-0130', '2021-0499', '2019-0050', '2020-0232', '2016-0752', '2019-0052', '2020-0385'}
IDs in Coded and Brady but absent from Gov: {'2020-0529', '2017-0215', '2020-0130', '2021-0499', '2019-0050', '2019-0052'}
IDs in Coded and Gov but absent from Brady: {'2019-0402', '2017-0115', '2016-0728', '2016-0481', '2021-0464', '2020-0479', '2019-0423', '2020-0177', '2020-0374', '2019-0541'}
Coded IDs not in either Brady or Gov: {'2016-009', '2019-0322', '2021-0569', '2016-240', '2020-0232', '2016-0752', '2020-0385'}
IDs in both Coded and Brady: {'2020-0162', '2016-0471', '2016-0173', '2016-0177', '2016-0532', '2018-0200', '2016-0684', '2021-0728', '2017-

In [519]:
coded_id_list = coded_tbl.tracking_id.to_list()


merged_tbl = brady_tbl[brady_tbl.tracking_id.isin(coded_id_list)]

# Convert tracking_id to string if it's not already, to ensure matching types
coded_tbl['tracking_id'] = coded_tbl['tracking_id'].astype(str)
merged_tbl['tracking_id'] = merged_tbl['tracking_id'].astype(str)

# Create a dictionary from coded_tbl for mapping
tracking_id_to_summary = dict(zip(coded_tbl['tracking_id'], coded_tbl['summary']))

# Map the summaries onto brady_tbl using the tracking_id column
merged_tbl['summary'] = merged_tbl['tracking_id'].map(tracking_id_to_summary)

merged_tbl.sort_values("tracking_id")

merged_tbl.disposition.unique()

# def clean_disposition(df):
#     df.loc[:, "disposition"] = (df.disposition.str.replace(r"^- ", "", regex=True)
#     )

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/1653337359.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tbl['tracking_id'] = merged_tbl['tracking_id'].astype(str)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/1653337359.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tbl['summary'] = merged_tbl['tracking_id'].map(tracking_id_to_summary)


array(['not sustained', 'unfounded', 'sustained - rui resign',
       'sustained rui resign', nan, 'sustained', 'nfim case',
       'rui-resigned under inves', 'exonerated', 'sustained - dismissed',
       '- unfounded', 'di-2', 'duplicate allegation',
       'pending investigation', '', 'duplicate investigation'],
      dtype=object)

In [520]:
merged_gov_tbl = gov_tbl[gov_tbl.tracking_id.isin(coded_id_list)]

def clean_demo(df):
    df.loc[:, "complainant_gender"] = (df.complainant_gender.fillna("")
                                        .str.replace(r"^f$", "female", regex=True)
                                        .str.replace(r"^m$", "male", regex=True)
                                        .str.replace(r"sex-unk", "", regex=True)
    )
    df.loc[:, "complainant_ethnicity"] = (df.complainant_ethnicity
                                          .str.replace(r"^w$", "white", regex=True)
                                          .str.replace(r"race-unknown", "", regex=True)
    )
    return df 

merged_gov_tbl = merged_gov_tbl.pipe(clean_demo)
merged_gov_tbl.columns

Index(['incident_type', 'complaint_tracking_number', 'date_complaint_occurred',
       'date_complaint_received_by_nopd_pib',
       'date_complaint_investigation_complete', 'complaint_classification',
       'investigation_status', 'disposition', 'bureau_of_complainant',
       'division_of_complainant', 'unit_of_complainant',
       'unit_additional_details_of_complainant',
       'working_status_of_complainant', 'shift_of_complainant',
       'rule_violation', 'paragraph_violation', 'unique_officer_allegation_id',
       'officer_race_ethnicity', 'officer_gender', 'officer_age',
       'officer_years_of_service', 'complainant_gender',
       'complainant_ethnicity', 'complainant_age', 'tracking_id'],
      dtype='object')

In [521]:
### age counts
def bucket_complainant_ages(gov_tbl):
    bins = [0, 18, 30, 40, float('inf')]  # float('inf') ensures all ages 41+ are included
    labels = ['0-18', '19-30', '31-40', '41+']
    
    gov_tbl['age_group'] = pd.cut(gov_tbl['complainant_age'], bins=bins, labels=labels, right=False)
    
    age_group_counts = gov_tbl['age_group'].value_counts()
    return age_group_counts

age_buckets = bucket_complainant_ages(merged_gov_tbl)

age_buckets

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/2093771939.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov_tbl['age_group'] = pd.cut(gov_tbl['complainant_age'], bins=bins, labels=labels, right=False)


age_group
19-30    36
41+      32
31-40    16
0-18      0
Name: count, dtype: int64

In [522]:
### demo counts 
def analyze_demographics(gov_tbl):
    def clean_demo(df):
        df.loc[:, "complainant_gender"] = (df.complainant_gender
                                            .str.replace(r"^f$", "female", regex=True)
                                            .str.replace(r"^m$", "male", regex=True)
        )
        df.loc[:, "complainant_ethnicity"] = df.complainant_ethnicity.str.replace(r"^w$", "white", regex=True)
        return df 
    
    cleaned_df = gov_tbl.pipe(clean_demo)

    final_df = cleaned_df[(cleaned_df['complainant_gender'] != '') & (cleaned_df['complainant_ethnicity'] != '')]

    grouped_counts = final_df.groupby(['complainant_gender', 'complainant_ethnicity']).size().reset_index(name='counts')

    total_counts = grouped_counts['counts'].sum()

    grouped_counts['percentage'] = (grouped_counts['counts'] / total_counts) * 100

    return grouped_counts

demo = analyze_demographics(merged_gov_tbl)

demo

,complainant_gender,complainant_ethnicity,counts,percentage
0,female,black,42,43.750000
1,female,hispanic,2,2.083333
2,female,white,19,19.791667
3,male,black,23,23.958333
4,male,hispanic,2,2.083333
5,male,white,8,8.333333


In [523]:

def analyze_demographics_with_age_buckets(gov_tbl):
    def clean_demo(df):
        df.loc[:, "complainant_gender"] = (df.complainant_gender
                                            .str.replace(r"^f$", "female", regex=True)
                                            .str.replace(r"^m$", "male", regex=True)
        )
        df.loc[:, "complainant_ethnicity"] = df.complainant_ethnicity.str.replace(r"^w$", "white", regex=True)
        return df
    
    # Define the bins and labels for age grouping
    bins = [0, 18, 30, 40, float('inf')]
    labels = ['0-18', '19-30', '31-40', '41+']
    
    cleaned_df = gov_tbl.pipe(clean_demo)

    filtered_df = cleaned_df[(cleaned_df['complainant_gender'] != '') & (cleaned_df['complainant_ethnicity'] != '')]

    filtered_df['age_group'] = pd.cut(filtered_df['complainant_age'], bins=bins, labels=labels, right=False)

    grouped_counts = filtered_df.groupby(['complainant_gender', 'complainant_ethnicity', 'age_group']).size().reset_index(name='counts')

    total_counts = grouped_counts['counts'].sum()
    grouped_counts['percentage'] = (grouped_counts['counts'] / total_counts) * 100

    return grouped_counts

demo_w_age = analyze_demographics_with_age_buckets(merged_gov_tbl)

demo_w_age

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/4123537522.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['age_group'] = pd.cut(filtered_df['complainant_age'], bins=bins, labels=labels, right=False)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_97787/4123537522.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_counts = filtered_df.groupby(['complainant_gender', 'complainant_ethnicity', 'age_group']).size().reset_index(name='counts')


,complainant_gender,complainant_ethnicity,age_group,counts,percentage
0,female,black,0-18,0,0.000000
1,female,black,19-30,12,15.189873
2,female,black,31-40,10,12.658228
3,female,black,41+,12,15.189873
4,female,hispanic,0-18,0,0.000000
5,female,hispanic,19-30,0,0.000000
6,female,hispanic,31-40,1,1.265823
7,female,hispanic,41+,0,0.000000
8,female,white,0-18,0,0.000000
9,female,white,19-30,6,7.594937


In [524]:
### number of missings that map to a code
### number of non-tags that map to a code
### take table of tags and use it to filter on the merged tbl 